<a href="https://colab.research.google.com/github/spencerklawans/466-Project/blob/main/WildfirePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wildfire Prediction
Brought to you by Gooogle Earth Engine

In [ ]:
# Import libraries.
import ee
import folium

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [78]:
#Init KDD libraries

%matplotlib inline

import numpy as np
import pandas as pd
import sklearn

In [79]:
California = ee.Geometry.Rectangle(-124.18, 41.39, -114.35, 32.75)

# Import our fire events as features
modis_fires = ee.FeatureCollection('JRC/GWIS/GlobFire/v2/FinalPerimeters').filterBounds(California)
# filter for only fires with greater than 10,000,000 square meters
modis_fires = modis_fires.filter(ee.Filter.gt('area', 10000000))

# Add a band for if this is a really big fire or not. 
# NOTE: We are using fire size as a proxy for severity
massive_fire_threshold = 100000000
# TODO: evaluate if this is an OK way of limiting (may be biased to north or south, or size, not sure how these are sorted before limit)
massive_fires = modis_fires.filter(ee.Filter.gt('area', massive_fire_threshold)).limit(10).map(lambda f: f.set({'massive': 1}))  # 158 fires
other_fires = modis_fires.filter(ee.Filter.lte('area', massive_fire_threshold)).limit(10).map(lambda f: f.set({'massive': 0}))   # 609 fires
modis_fires = massive_fires.merge(other_fires)

In [80]:
slo = ee.Geometry.Point([-120.4358, 35.3102])

# Import ImageCollections
landfire_prs = ee.Image("LANDFIRE/Fire/PRS/v1_2_0/CONUS")
landfire_evc = ee.Image("LANDFIRE/Vegetation/EVC/v1_4_0/CONUS")
landfire_evh = ee.Image("LANDFIRE/Vegetation/EVH/v1_4_0/CONUS")
landfire_evt = ee.Image("LANDFIRE/Vegetation/EVT/v1_4_0/CONUS")
jaxa_temp = ee.ImageCollection("JAXA/GCOM-C/L3/LAND/LST/V1")

composite = landfire_evc.addBands(landfire_prs).addBands(landfire_evh).addBands(landfire_evt)

In [82]:
n_train = 0.5
n_val = 0.5
bands = ['PRS', 'EVC','EVH','EVT','LST_AVE']
# split the data into training and test sets
fire_image = ee.Image().byte().paint(modis_fires, 'massive').rename('massive')
# fire_image2 = ee.Image().paint(modis_fires,'IntitalDate').rename('InitialDate')

fire_image2 = modis_fires.reduceToImage(
    properties = ['InitialDate'], 
    reducer = ee.Reducer.first())

composite = composite.addBands(fire_image2)





#fire_image = ee.Image().constant(modis_fires.first().toDictionary(props).values()).rename(props)

#fire_image = ee.ImageCollection(modis_fires.map(lambda x :ee.Image().constant(x.toDictionary(props).values()).rename(props)))

#print(fire_image.first().getInfo())
# final = composite.addBands(fire_image)
# fire_image.getInfo()

# final.getInfo()
# data = composite.select(bands).sampleRegions(collection=modis_fires,
#                                              properties=['massive'],
#                                              scale=30,
#                                              tileScale=16,
#                                              projection='EPSG:3665',
#                                              region=modis_fires.geometry())
fire_points = fire_image.addBands(ee.Image.pixelLonLat()).stratifiedSample(
                                                              numPoints=10,
                                                              classBand='massive',
                                                              scale=30,
                                                              region=modis_fires.geometry()).map(
                                                                  lambda f: f.setGeometry(ee.Geometry.Point([f.get('longitude'), f.get('latitude')])))

                                                              



data = composite.sampleRegions(fire_points, properties=['massive'], scale=30, geometries=True)
data = data.randomColumn(seed=0)

# def getTempForDate(feature):
#   date_filter = ee.Filter.date(0, feature.get('first'))
#   tempImage = jaxa_temp.filter(date_filter).limit(1, 'system:time_start', False).first()
#   return feature.set({"jaxa_temp": tempImage.sample(feature.geometry()).first().get('LST_AVE')})

# data = data.map(getTempForDate)

#massive_fires = modis_fires.filter(ee.Filter.gt('area', massive_fire_threshold)).limit(10).map(lambda f: f.set({'massive': 1})) 

temp = data.filter(ee.Filter.lt('random', 1-n_train))
training = data.filter(ee.Filter.gte('random', 1-n_train))
validation = temp.filter(ee.Filter.lt('random', (1-n_train)*n_val))
testing = temp.filter(ee.Filter.gte('random', (1-n_train)*n_val))
#fire_image.getInfo()
#data.first().getInfo()
#fire_image2.getInfo()
#modis_fires.get('InitialDate').getInfo()

#images = modis_fires.map(
    #lambda f: ee.Image(ee.Number(f.get('InitialDate'))).rename('InitialDate'))

#images.first().getInfo()
#modis_fires.first().getInfo()


# images.getInfo()
#data.getInfo()

In [ ]:
#small_points = composite.sampleRegions(other_fires, geometries=True)
classifier = ee.Classifier.smileNaiveBayes().train(features=training, classProperty='massive', inputProperties=bands)
pred = validation.classify(classifier)
pred.first().getInfo()
#pred.errorMatrix('massive', 'classification').accuracy().getInfo()

In [ ]:
# Set visualization parameters. I'm pretty sure min and max should track to the min and max values of the set at https://developers.google.com/earth-engine/datasets/catalog/LANDFIRE_Fire_PRS_v1_2_0#bands
prs_viz_params = {
    'bands': ['PRS']
}

evc_vis_params = {
    'bands': ['EVC'],
    'opacity': 0.5
}
fires_viz_params = {
    'bands': ['massive'],
    'palette': ['#FF0000', '#0000FF'],
     'min': 0,
     'max': 1,
     'opacity': 0.5
}
pred_viz_params = {
    'palette': ['#FF0000', '#0000FF'],
    'min': 0,
    'max': 1,
    'opacity': 0.5
}

# Create a folium map object.
my_map = folium.Map(location=[35.3102, -120.4358], zoom_start=8)

# Add the fire severity & existing vegetation type images to the map object.
my_map.add_ee_layer(landfire_prs, prs_viz_params, 'Percent Replacement-Severity Fire')
my_map.add_ee_layer(landfire_evc, evc_vis_params, 'Existing Vegetation Cover')
my_map.add_ee_layer(fire_image, fires_viz_params, 'Fire Events')
my_map.add_ee_layer(images, {"bands": ['first']}, 'ImageDate')
#my_map.add_ee_layer(ee.Image().paint(pred, 'classification'), fires_viz_params, 'Predicted Events')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)